# Prática com Sistemas de Recomendação
---
## Criar Sistemas de recomendação de jogos
*Classificação de jogos com base de dados da steam*


# Algoritmo

In [1]:
import pandas as pd
from datetime import datetime

# Carregar o conjunto de dados a partir do link direto
url = 'https://drive.google.com/uc?export=download&id=1SdCK7a5E9_vQKTEmMtEROrq-LBsKlTPE'
data = pd.read_csv(url, dtype={'id': str}, low_memory=False)

# Pré-processamento dos dados
data['positive_ratings'] = pd.to_numeric(data['positive_ratings'], errors='coerce')
data['negative_ratings'] = pd.to_numeric(data['negative_ratings'], errors='coerce')

# Tratamento de dados ausentes
data.dropna(subset=['positive_ratings', 'negative_ratings'], inplace=True)

# Definir o pré-requisito (mínimo de votos)
m = data['positive_ratings'].quantile(0.70)

# Filtrar os jogos que atendem ao pré-requisito
filtered_games = data[data['positive_ratings'] >= m]

# Calcular a média das avaliações de todos os jogos
mean_rating = data['positive_ratings'].mean()

# Calcular a pontuação para cada jogo
def calculate_score(row):
    v = row['positive_ratings']
    r = row['negative_ratings']
    return (v / (v + m) * r) + (m / (m + v) * mean_rating)

# Aplicar a função para calcular a pontuação de cada jogo
filtered_games = filtered_games.copy()
filtered_games['score'] = filtered_games.apply(calculate_score, axis=1)

In [2]:
# Função para obter as recomendações com base em um jogo específico, gênero ou outros parâmetros
def get_game_recommendations(title=None, genre=None, top_n=50, price=None, release_date=None, platforms=None):
    # Filtrar por título, gênero e/ou outros parâmetros fornecidos
    games = filtered_games.copy()

    if title:
        games = games[games['name'].str.contains(title, case=False)]
    
    if genre:
        games = games[games['genres'].str.contains(genre, case=False)]
    
    # Filtrar por preço, se fornecido
    if price is not None:
        games = games[games['price'] <= price]
    
    # Filtrar por data de lançamento, se fornecida
    if release_date is not None:
        release_date = datetime.strptime(release_date, '%Y-%m-%d').date()
        games = games[pd.to_datetime(games['release_date']).dt.date >= release_date]
    
    # Filtrar por plataforma, se fornecida
    if platforms:
        games = games[games['platforms'].str.contains(platforms, case=False)]
    
    # Ordenar por pontuação em ordem decrescente
    games = games.sort_values('score', ascending=False)
    
    # Obter as top_n recomendações
    recommendations = games[['name', 'price', 'release_date', 'platforms', 'genres', 'positive_ratings', 'negative_ratings', 'score']].head(top_n)
    
    # Aplicar estilos ao DataFrame de recomendações
    styled_recommendations = recommendations.style\
        .set_properties(**{'text-align': 'left'})
    
    # Retornar o DataFrame de recomendações com estilos
    return styled_recommendations


# Testes

In [3]:
# Teste de busca por título
get_game_recommendations(title='Grand Theft Auto', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
2478,Grand Theft Auto V,24.990000,2015-04-13,windows,Action;Adventure,329061,139308,139273.123114
297,Grand Theft Auto IV,19.990000,2008-12-02,windows,Action;Adventure,35240,18926,18883.879805
290,Grand Theft Auto: San Andreas,9.990000,2008-01-04,windows,Action,26877,3243,3236.096341
298,Grand Theft Auto: Episodes from Liberty City,14.990000,2010-04-12,windows,Action,6401,2345,2327.790154
289,Grand Theft Auto: Vice City,5.990000,2011-01-10,windows,Action,9817,819,820.522157
288,Grand Theft Auto III,5.990000,2008-01-04,windows,Action,4718,750,754.331672
295,Grand Theft Auto 2,0.000000,2008-01-04,windows,Action,184,17,322.750402
294,Grand Theft Auto,0.000000,2008-01-04,windows,Action,199,30,315.660842


In [4]:
# Teste de busca por gênero
get_game_recommendations(genre='RPG', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
2016,Rust,27.790000,2018-02-08,windows;mac,Action;Adventure;Indie;Massively Multiplayer;RPG,220370,53709,53689.155404
5916,Fallout 4,19.990000,2015-11-09,windows,RPG,110376,45377,45343.655097
3251,Robocraft,0.000000,2017-08-24,windows;mac;linux,Action;Free to Play;Indie;Massively Multiplayer;RPG;Simulation,80360,30987,30956.060395
5472,Tom Clancy’s The Division™,41.990000,2016-03-07,windows,Action;Adventure;RPG,49832,25607,25566.083750
1982,7 Days to Die,18.990000,2013-12-13,windows;mac;linux,Nudity;Violent;Gore;Action;Adventure;Indie;RPG;Simulation;Strategy;Early Access,60718,16905,16883.288702
21463,ATLAS,23.790000,2018-12-22,windows,Action;Adventure;Massively Multiplayer;RPG;Early Access,8621,16359,16212.544274
1025,The Elder Scrolls V: Skyrim,9.990000,2011-11-10,windows,RPG,237303,14951,14946.122347
9534,The Elder Scrolls V: Skyrim Special Edition,29.990000,2016-10-27,windows,RPG,47749,14719,14695.195212


In [5]:
# Teste de busca por preço
get_game_recommendations(price=20, top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
7809,Z1 Battle Royale,0.000000,2018-02-28,windows,Action;Adventure;Free to Play;Massively Multiplayer,111434,91664,91596.520910
5235,Tom Clancy's Rainbow Six® Siege,16.990000,2015-12-01,windows,Action,251178,59620,59600.636017
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
5916,Fallout 4,19.990000,2015-11-09,windows,RPG,110376,45377,45343.655097
6046,Dead by Daylight,14.990000,2016-06-14,windows,Action,140721,44985,44959.072408
1607,Heroes & Generals,0.000000,2016-10-18,windows,Action;Free to Play;Indie;Massively Multiplayer,65856,34416,34373.938676
19,Team Fortress 2,0.000000,2007-10-10,windows;mac;linux,Action;Free to Play,515879,34036,34030.685768
8129,Paladins®,0.000000,2018-05-08,windows,Action;Free to Play,169580,32407,32391.635810


In [6]:
# Teste de busca por data de lançamento
get_game_recommendations(release_date='2010-01-01', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,26.990000,2017-12-21,windows,Action;Adventure;Massively Multiplayer,496184,487076,486994.704525
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
2478,Grand Theft Auto V,24.990000,2015-04-13,windows,Action;Adventure,329061,139308,139273.123114
7809,Z1 Battle Royale,0.000000,2018-02-28,windows,Action;Adventure;Free to Play;Massively Multiplayer,111434,91664,91596.520910
1506,DayZ,33.990000,2018-12-13,windows,Action;Adventure;Massively Multiplayer,118435,77169,77115.658055
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
5235,Tom Clancy's Rainbow Six® Siege,16.990000,2015-12-01,windows,Action,251178,59620,59600.636017
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
2563,No Man's Sky,39.990000,2016-08-12,windows,Action;Adventure,63444,56488,56415.503933


In [7]:
# Teste de busca por plataforma (Windows)
get_game_recommendations(platforms='Windows', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,26.990000,2017-12-21,windows,Action;Adventure;Massively Multiplayer,496184,487076,486994.704525
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
2478,Grand Theft Auto V,24.990000,2015-04-13,windows,Action;Adventure,329061,139308,139273.123114
7809,Z1 Battle Royale,0.000000,2018-02-28,windows,Action;Adventure;Free to Play;Massively Multiplayer,111434,91664,91596.520910
1506,DayZ,33.990000,2018-12-13,windows,Action;Adventure;Massively Multiplayer,118435,77169,77115.658055
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
5235,Tom Clancy's Rainbow Six® Siege,16.990000,2015-12-01,windows,Action,251178,59620,59600.636017
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
2563,No Man's Sky,39.990000,2016-08-12,windows,Action;Adventure,63444,56488,56415.503933


In [8]:
# Teste de busca por plataforma (Linux)
get_game_recommendations(platforms='Linux', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
19,Team Fortress 2,0.000000,2007-10-10,windows;mac;linux,Action;Free to Play,515879,34036,34030.685768
3362,Unturned,0.000000,2017-07-07,windows;mac;linux,Action;Adventure;Casual;Free to Play;Indie,292574,31482,31473.355205
3251,Robocraft,0.000000,2017-08-24,windows;mac;linux,Action;Free to Play;Indie;Massively Multiplayer;RPG;Simulation,80360,30987,30956.060395
1727,War Thunder,0.000000,2013-08-15,windows;mac;linux,Action;Free to Play;Massively Multiplayer;Simulation,83852,22166,22145.070332
2031,Rocket League®,14.990000,2015-07-07,windows;mac;linux,Action;Indie;Racing;Sports,242561,21816,21808.879768
2878,Sid Meier’s Civilization® VI,49.990000,2016-10-20,windows;mac;linux,Strategy,39127,18325,18288.327502


In [9]:
# Teste de busca por plataforma (Mac)
get_game_recommendations(platforms='Mac', top_n=10)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
2016,Rust,27.790000,2018-02-08,windows;mac,Action;Adventure;Indie;Massively Multiplayer;RPG,220370,53709,53689.155404
19,Team Fortress 2,0.000000,2007-10-10,windows;mac;linux,Action;Free to Play,515879,34036,34030.685768
3362,Unturned,0.000000,2017-07-07,windows;mac;linux,Action;Adventure;Casual;Free to Play;Indie,292574,31482,31473.355205
3251,Robocraft,0.000000,2017-08-24,windows;mac;linux,Action;Free to Play;Indie;Massively Multiplayer;RPG;Simulation,80360,30987,30956.060395
3504,Call of Duty®: Black Ops III,44.990000,2015-11-05,windows;mac,Action;Adventure,36255,23157,23106.392244
1727,War Thunder,0.000000,2013-08-15,windows;mac;linux,Action;Free to Play;Massively Multiplayer;Simulation,83852,22166,22145.070332
2031,Rocket League®,14.990000,2015-07-07,windows;mac;linux,Action;Indie;Racing;Sports,242561,21816,21808.879768


In [10]:
# Teste com múltiplos parâmetros de busca
get_game_recommendations(genre='Action', price=30, platforms='Windows')

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,26.990000,2017-12-21,windows,Action;Adventure;Massively Multiplayer,496184,487076,486994.704525
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
2478,Grand Theft Auto V,24.990000,2015-04-13,windows,Action;Adventure,329061,139308,139273.123114
7809,Z1 Battle Royale,0.000000,2018-02-28,windows,Action;Adventure;Free to Play;Massively Multiplayer,111434,91664,91596.520910
5235,Tom Clancy's Rainbow Six® Siege,16.990000,2015-12-01,windows,Action,251178,59620,59600.636017
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
2016,Rust,27.790000,2018-02-08,windows;mac,Action;Adventure;Indie;Massively Multiplayer;RPG,220370,53709,53689.155404
6046,Dead by Daylight,14.990000,2016-06-14,windows,Action,140721,44985,44959.072408
1607,Heroes & Generals,0.000000,2016-10-18,windows,Action;Free to Play;Indie;Massively Multiplayer,65856,34416,34373.938676
